In [ ]:
from cloth_tools.dataset.download import get_latest_observation_start_url

server_url = "https://robotlab.ugent.be"

get_latest_observation_start_url(server_url) + "/image_left.png"

In [ ]:
from pathlib import Path
from cloth_tools.dataset.download import download_latest_observation

data_dir = Path("data")
dataset_dir = data_dir / "downloaded_dataset_0000"

observation_dir, sample_id = download_latest_observation(dataset_dir, server_url)
print(observation_dir)
print(sample_id)

In [ ]:
from cloth_tools.dataset.format import load_competition_observation

observation = load_competition_observation(observation_dir)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(observation.image_left)
plt.title("Left image")
plt.subplot(1, 2, 2)
plt.imshow(observation.image_right)
plt.title("Right image")
plt.show()

In [ ]:
from cloth_tools.dataset.bookkeeping import datetime_for_filename

datetime_for_filename()

In [ ]:
from cloth_tools.annotation.grasp_annotation import grasp_hanging_cloth_pose
import numpy as np

grasp_pose_fixed = grasp_hanging_cloth_pose(np.array([0, 0, 0.5]), np.array([1, 0, 0]), 0.0)
grasp_pose_fixed

In [ ]:
import cv2
from cloth_tools.visualization.opencv import draw_pose

X_W_C = observation.camera_pose_in_world
intrinsics = observation.camera_intrinsics

image_bgr = cv2.cvtColor(observation.image_left, cv2.COLOR_RGB2BGR)
draw_pose(image_bgr, grasp_pose_fixed, intrinsics, X_W_C, 0.1)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 5))
plt.imshow(image_rgb)
plt.title("Example grasp pose")
plt.show()


In [ ]:
import os
grasps_dir = f"data/grasps_{sample_id}"

os.makedirs(grasps_dir, exist_ok=True)

In [ ]:
grasp_pose_name = f"grasp_pose_{datetime_for_filename()}.json"
grasp_pose_name

In [ ]:
import json
from airo_dataset_tools.data_parsers.pose import Pose

grasp_pose_file = os.path.join(grasps_dir, grasp_pose_name)

with open(grasp_pose_file, "w") as f:
    grasp_pose_model = Pose.from_homogeneous_matrix(grasp_pose_fixed)
    json.dump(grasp_pose_model.model_dump(exclude_none=False), f, indent=4)

os.listdir(grasps_dir)

In [ ]:
from cloth_tools.dataset.upload import upload_grasp

# Example Usage
team_name = "dummy_team"
upload_grasp(grasp_pose_file, team_name, sample_id, server_url) 